In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 
import keras
from keras.layers import Dense, LSTM, SimpleRNN, GRU, Input, Conv1D, MaxPooling1D, Flatten
from keras.models import Model, Sequential  

df_train = pd.read_csv("sales_train_validation.csv") 
#df_sell = pd.read_csv("./drive/My Drive/sell_prices.csv")
#df_calendar = pd.read_csv("./drive/My Drive/calendar.csv")
#df_ss = pd.read_csv("./drive/My Drive/sample_submission_m5.csv")


Using TensorFlow backend.


In [2]:
days = range(1,1913+1) 
time_series_cols = [] 
for i in days: 
    time_series_cols.append('d_'+str(i)) 
time_series_cols[:10]

['d_1', 'd_2', 'd_3', 'd_4', 'd_5', 'd_6', 'd_7', 'd_8', 'd_9', 'd_10']

In [3]:
time_series_data = df_train[time_series_cols] 
time_series_data

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,0,0,0,0,0,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,0,0,0,0,0,0,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,0,0,0,0,0,0,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,0,0,2,2,0,3,1,4,1,0,...,2,0,0,0,0,0,1,0,0,1
30486,0,0,0,0,0,5,0,1,1,3,...,0,0,0,0,0,0,0,0,1,0
30487,0,6,0,2,2,4,1,8,5,2,...,2,1,0,2,0,1,0,0,1,0
30488,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [4]:
time_series_data = np.asarray(time_series_data).astype('float32') 
print(time_series_data.shape)

(30490, 1913)


In [5]:
def build_model(timesteps, features): 
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=7, activation='relu', input_shape=(timesteps, features), padding='same'))
    model.add(MaxPooling1D(pool_size=7))
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(features))
    return model  

In [6]:
def split_xy2(dataset, time_steps):
    x,y = list(), list() 
    for i in range(0,dataset.shape[1]-time_steps): 
        x.append(dataset[:,i:i+time_steps]) 
        y.append(dataset[:,i+time_steps]) 
    return np.array(x), np.array(y) 

In [7]:
predictions = [] 

for i in range(1,29): 
    print("day {}".format(i)) 
    data = time_series_data[:,::i]
    x_train, y_train = split_xy2(data,28) 
    x_pred = data[:,data.shape[1]-28:] 
    x_train = x_train.reshape(x_train.shape[0],28,30490)  
    x_pred = x_pred.reshape(1,28,30490)
    print(x_train.shape) 
    print(x_pred.shape)  
    model = build_model(28,30490)    
    model.compile(optimizer='adam',loss='mse')  
    model.fit(x_train,y_train,epochs=150)   
    pred = model.predict(x_pred) 
    print(pred) 
    predictions.append(pred) 


day 1
(1885, 28, 30490)
(1, 28, 30490)
Epoch 1/150
1885/1885 [==============================] - 13s 7ms/step - loss: 10.7760
Epoch 2/150
1885/1885 [==============================] - 11s 6ms/step - loss: 6.8834
Epoch 3/150
1885/1885 [==============================] - 11s 6ms/step - loss: 6.5485
Epoch 4/150
1885/1885 [==============================] - 11s 6ms/step - loss: 6.2680
Epoch 5/150
1885/1885 [==============================] - 11s 6ms/step - loss: 6.0251
Epoch 6/150
1885/1885 [==============================] - 11s 6ms/step - loss: 5.8738
Epoch 7/150
1885/1885 [==============================] - 11s 6ms/step - loss: 5.7928
Epoch 8/150
1885/1885 [==============================] - 11s 6ms/step - loss: 5.7346
Epoch 9/150
1885/1885 [==============================] - 11s 6ms/step - loss: 5.8708
Epoch 10/150
1885/1885 [==============================] - 11s 6ms/step - loss: 5.7681
Epoch 11/150
1885/1885 [==============================] - 11s 6ms/step - loss: 5.6507
Epoch 12/150
1885/1885 

In [8]:
predictions = np.asarray(predictions) 
print(predictions.shape)

(28, 1, 30490)


In [9]:
# flatten each of predictions[i] 
pred_flattened = [] 
for i in range(28): 
    pred_flattened.append(predictions[i].flatten()) 

In [10]:
# checking if shape is flattened 
pred_flattened[0]

array([0.6527668 , 0.09848265, 0.51798934, ..., 0.48384723, 0.72201407,
       1.4747326 ], dtype=float32)

In [11]:
cols = [] 
for i in range(1,29): 
    cols.append("F"+str(i)) 
df = pd.DataFrame(pred_flattened)   
df = df.T # transpose 
df.columns = cols # rename column 
df 

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,0.652767,0.321678,0.450631,0.584343,0.691300,0.686568,0.975437,0.654573,0.682969,0.627600,...,0.413576,0.827276,1.305531,0.387399,0.695094,0.999675,0.661918,0.485207,1.390636,1.225675
1,0.098483,0.213341,0.432739,0.156725,0.526906,0.085133,0.107766,0.467761,0.292426,0.584577,...,0.804833,0.068974,0.899943,0.755831,0.336254,0.512544,0.380866,0.272177,-0.028911,0.041769
2,0.517989,0.343414,0.352255,0.311222,0.644165,0.269279,0.408120,0.006066,0.452284,0.311528,...,1.022859,0.565381,1.266871,0.471243,0.440960,0.315260,0.290942,0.224949,0.838773,0.894257
3,2.101281,1.628847,1.700554,1.769646,2.066584,1.180008,2.043008,2.872483,1.643915,1.387990,...,2.232034,2.127364,2.990549,1.866175,2.125758,2.751513,1.936637,1.782168,3.023649,2.759682
4,1.046804,1.019761,0.633501,1.125210,1.175162,-0.016944,1.199389,1.357964,1.600051,0.946401,...,1.513074,0.918700,1.244649,1.697127,1.175851,1.004789,1.266102,1.037344,1.303569,0.511514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,0.607732,0.423440,0.289453,0.566819,0.405571,0.491017,0.668292,0.545760,0.490544,0.400311,...,0.433271,0.411842,0.578740,0.660257,0.374819,0.592676,0.346781,-0.249950,0.296282,1.050632
30486,-0.011917,0.028451,-0.002255,0.029083,-0.124626,-0.130349,-0.091943,0.002490,0.137396,-0.019682,...,0.124759,-0.009434,0.207572,0.236018,-0.124930,0.112482,0.212719,0.278144,-0.101422,0.013568
30487,0.483847,0.642229,0.756216,0.693061,0.605015,1.200222,0.918825,0.490670,0.734682,0.314756,...,0.124315,0.607843,0.698102,0.350739,0.419708,0.710799,0.329826,0.987689,0.783403,1.109580
30488,0.722014,0.828928,0.606185,0.791924,0.773190,1.176511,0.660795,0.871163,0.900244,0.771857,...,0.673277,0.664371,0.848436,0.759499,0.601226,0.977673,0.662333,1.483587,0.811099,0.727372


In [12]:
validation_ids = df_train['id'].values
evaluation_ids = [i.replace('validation', 'evaluation') for i in validation_ids]
ids = np.concatenate([validation_ids, evaluation_ids])       
ids

array(['HOBBIES_1_001_CA_1_validation', 'HOBBIES_1_002_CA_1_validation',
       'HOBBIES_1_003_CA_1_validation', ...,
       'FOODS_3_825_WI_3_evaluation', 'FOODS_3_826_WI_3_evaluation',
       'FOODS_3_827_WI_3_evaluation'], dtype=object)

In [13]:
sub_df = pd.DataFrame(ids, columns=['id']) 
df = pd.concat([df]*2).reset_index(drop=True) 
sub_df = pd.concat([sub_df,df],axis=1) 
sub_df.to_csv('Conv1D_28.csv',index=False)  

In [14]:
sub_df

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.652767,0.321678,0.450631,0.584343,0.691300,0.686568,0.975437,0.654573,0.682969,...,0.413576,0.827276,1.305531,0.387399,0.695094,0.999675,0.661918,0.485207,1.390636,1.225675
1,HOBBIES_1_002_CA_1_validation,0.098483,0.213341,0.432739,0.156725,0.526906,0.085133,0.107766,0.467761,0.292426,...,0.804833,0.068974,0.899943,0.755831,0.336254,0.512544,0.380866,0.272177,-0.028911,0.041769
2,HOBBIES_1_003_CA_1_validation,0.517989,0.343414,0.352255,0.311222,0.644165,0.269279,0.408120,0.006066,0.452284,...,1.022859,0.565381,1.266871,0.471243,0.440960,0.315260,0.290942,0.224949,0.838773,0.894257
3,HOBBIES_1_004_CA_1_validation,2.101281,1.628847,1.700554,1.769646,2.066584,1.180008,2.043008,2.872483,1.643915,...,2.232034,2.127364,2.990549,1.866175,2.125758,2.751513,1.936637,1.782168,3.023649,2.759682
4,HOBBIES_1_005_CA_1_validation,1.046804,1.019761,0.633501,1.125210,1.175162,-0.016944,1.199389,1.357964,1.600051,...,1.513074,0.918700,1.244649,1.697127,1.175851,1.004789,1.266102,1.037344,1.303569,0.511514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.607732,0.423440,0.289453,0.566819,0.405571,0.491017,0.668292,0.545760,0.490544,...,0.433271,0.411842,0.578740,0.660257,0.374819,0.592676,0.346781,-0.249950,0.296282,1.050632
60976,FOODS_3_824_WI_3_evaluation,-0.011917,0.028451,-0.002255,0.029083,-0.124626,-0.130349,-0.091943,0.002490,0.137396,...,0.124759,-0.009434,0.207572,0.236018,-0.124930,0.112482,0.212719,0.278144,-0.101422,0.013568
60977,FOODS_3_825_WI_3_evaluation,0.483847,0.642229,0.756216,0.693061,0.605015,1.200222,0.918825,0.490670,0.734682,...,0.124315,0.607843,0.698102,0.350739,0.419708,0.710799,0.329826,0.987689,0.783403,1.109580
60978,FOODS_3_826_WI_3_evaluation,0.722014,0.828928,0.606185,0.791924,0.773190,1.176511,0.660795,0.871163,0.900244,...,0.673277,0.664371,0.848436,0.759499,0.601226,0.977673,0.662333,1.483587,0.811099,0.727372
